Alex Thornton (apt2141)  
Columbia University Fall 2021  
ELEN E6893 - Big Data Analytics  
Research Project: SpotifyClassifier

1. Connect to GCP

In [3]:
!gcloud auth login
!gcloud config set project spotifyclassifier

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=LWaJ5EVu85A7Cmv8JefKWmZnNqKEiN&prompt=consent&access_type=offline&code_challenge=ZGSEdIoU0oIG3C-uyJREDH6SOA4zRxpOcJlDaCIJgzg&code_challenge_method=S256

Enter verification code: 4/1AX4XfWhamtD-Q2riDD057kxO6EomdbfpFUH_9kuSsqwHHJkoa8LJZRiebCo

You are now logged in as [apt2141@columbia.edu].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID
Updated property [core/project].
E: Unable to locate package gcsfuse


In [6]:
#Upgrade cloud machine to have package for mounting bucket
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   103k      0 --:--:-- --:--:-- --:--:--  103k
OK
41 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 11.3 MB of archives.
After this operation, 24.0 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.37.0_amd64.deb ...
Unpacking gcsfuse (0.37.0) ...
Setting up gcsfuse (0.37.0) ...


In [9]:
#Mount cloud bucket locally
!mkdir /mnt/track-lists
!gcsfuse gs://track-lists /mnt/track-lists

2021/11/29 04:54:04.915322 Start gcsfuse/0.37.0 (Go version go1.17.2) for app "" using mount point: /mnt/track-lists
2021/11/29 04:54:04.924680 Opening GCS connection...
2021/11/29 04:54:44.378437 Mounting file system "gs://track-lists"...
2021/11/29 04:54:44.380270 File system has been successfully mounted.


2. Setup Spotify API

In [ ]:
token = "SECRET TOKEN"

3. Import Packages

In [ ]:
import requests
import json

1. Data loading